# Analysis with CSV Extracts

## This analysis identifies all patients with Prediabetes as well as those within the past year

In [30]:
# Step 1: Import Necessary Libraries
import pandas as pd
from datetime import datetime

# Step 2: Load Data
conditions_path = "/home/cdsw/exported-data/hl7_condition.csv"
conditions_df = pd.read_csv(conditions_path, quotechar='"')

# Load the patient metadata
patient_pii_path = "/home/cdsw/exported-data/hl7_patient_pii.csv"
patient_metadata_df = pd.read_csv(patient_pii_path, quotechar='"')

# Step 3: Data Preprocessing
def preprocess_data(df):
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].str.strip('"').str.strip('\'')
    return df

conditions_df = preprocess_data(conditions_df)

### Identify all patients with prediabetes

In [32]:
# Convert 'onsetdatetime' from string to datetime and make it timezone aware
conditions_df['onsetdatetime'] = pd.to_datetime(conditions_df['onsetdatetime'], errors='coerce', utc=True)

# Filter for Prediabetes conditions only, removing the date restriction
conditions_df = conditions_df[
    (conditions_df['condition_display'] == "Prediabetes")
]

# Preprocess the patient metadata (ensure correct data types and stripping extra quotes)
patient_metadata_df = patient_metadata_df[['fullUrl', 'resource_name_0_family', 'resource_name_0_given_0']]

# Join the dataframes
prediabetes_patient_info_df = conditions_df.merge(
    patient_metadata_df,
    left_on='subject_reference',
    right_on='fullUrl',
    how='left'
)

# Make a copy to avoid SettingWithCopyWarning when renaming columns
final_df = prediabetes_patient_info_df[['condition_display', 'subject_reference', 'onsetdatetime', 'encounter_reference', 'resource_name_0_given_0', 'resource_name_0_family']].copy()
final_df.rename(columns={
    'resource_name_0_given_0': 'First Name',
    'resource_name_0_family': 'Last Name'
}, inplace=True)

# Display the final dataframe
print(final_df)

   condition_display                              subject_reference  \
0        Prediabetes  urn:uuid:2ee443fc-63eb-1706-3864-cc2646839e19   
1        Prediabetes  urn:uuid:05c4608d-bd9a-5d04-41d7-a0293da7f5a5   
2        Prediabetes  urn:uuid:35b7931d-e61b-9c71-7c78-26e97b6de524   
3        Prediabetes  urn:uuid:1ffbf770-5e69-f93e-916c-a316002eead3   
4        Prediabetes  urn:uuid:29257306-9929-b1f9-49f8-ac43800341fe   
5        Prediabetes  urn:uuid:b6d3e48c-cb6a-5e32-167e-9dc2ebc0a007   
6        Prediabetes  urn:uuid:0b595f34-417b-c1ca-6e47-5c9bc7d8f58e   
7        Prediabetes  urn:uuid:090a02d7-c906-58e1-8697-046720cd40a7   
8        Prediabetes  urn:uuid:eaa23cfe-4255-47fd-d741-b65bb704b8d2   
9        Prediabetes  urn:uuid:54ac079a-90af-6fbd-bd78-34bcbc8308f3   
10       Prediabetes  urn:uuid:e58f8f49-70af-e8fb-f52f-0d02b0b1fefd   
11       Prediabetes  urn:uuid:dbbdea19-8376-10e8-c104-f23fe8fbf3fd   
12       Prediabetes  urn:uuid:2b1f2ab3-4e87-85d1-304f-10250f4d8faf   
13    

### Identify patients with prediabetes in the past year

In [33]:
# Convert 'onsetdatetime' from string to datetime and make it timezone aware
conditions_df['onsetdatetime'] = pd.to_datetime(conditions_df['onsetdatetime'], errors='coerce', utc=True)
current_date = pd.Timestamp.utcnow()
one_year_ago = current_date - pd.DateOffset(years=1)
conditions_df = conditions_df[
    (conditions_df['condition_display'] == "Prediabetes") &
    (conditions_df['onsetdatetime'] >= one_year_ago)
]

# # Preprocess the patient metadata (ensure correct data types and stripping extra quotes)
patient_metadata_df = patient_metadata_df[['fullUrl', 'resource_name_0_family', 'resource_name_0_given_0']]

# Join the dataframes
prediabetes_patient_info_df = conditions_df.merge(
    patient_metadata_df,
    left_on='subject_reference',
    right_on='fullUrl',
    how='left'
)

# Make a copy to avoid SettingWithCopyWarning when renaming columns
final_df = prediabetes_patient_info_df[['condition_display', 'subject_reference', 'onsetdatetime', 'encounter_reference', 'resource_name_0_given_0', 'resource_name_0_family']].copy()
final_df.rename(columns={
    'resource_name_0_given_0': 'First Name',
    'resource_name_0_family': 'Last Name'
}, inplace=True)

# Display the final dataframe
print(final_df)

  condition_display                              subject_reference  \
0       Prediabetes  urn:uuid:29257306-9929-b1f9-49f8-ac43800341fe   
1       Prediabetes  urn:uuid:ce91b7b5-7064-8e8e-6327-3d836b2e180a   
2       Prediabetes  urn:uuid:2703570e-f634-d4cc-13db-2ae14e19ae59   

              onsetdatetime                            encounter_reference  \
0 2023-09-26 18:21:43+00:00  urn:uuid:56c54261-78c0-6552-77bf-820a14ffc0ec   
1 2023-11-14 08:50:58+00:00  urn:uuid:06d3dc30-0024-a472-990c-761ad33306d5   
2 2024-03-23 00:34:37+00:00  urn:uuid:a7c99e0b-4c5b-1749-4d8d-870b2e885463   

  First Name    Last Name  
0    Chet188     Bruen238  
1   Carin553  Kassulke119  
2   Norma469   Langosh790  
